In [17]:
##############
#Dependencies#
##############
%matplotlib notebook
import datetime
import pandas
import numpy as np
import folium


In [22]:

route_df= pandas.read_csv('Data/route_summary.csv')
route_df = route_df.set_index("Station")

route_df= route_df.fillna(0)
route_df

,Unnamed: 0,Heading,Time,Order,Speed,Flow,Lat,Lng
Station,,,,,,,,
44.89021,0,-93.27616,0,0.0,0.0,0.0,0.00000,0.00000
44.89021,1,-93.27616,0,0.0,0.0,0.0,0.00000,0.00000
44.89021,2,-93.27616,0,0.0,0.0,0.0,0.00000,0.00000
S10,3,SB,Wed Oct 10 01:15:59 CDT 2018,54.0,40.0,60.0,44.92003,-93.27488
S10,4,SB,Wed Oct 10 01:15:59 CDT 2018,54.0,40.0,60.0,44.92003,-93.27488
S10,5,SB,Wed Oct 10 01:15:59 CDT 2018,54.0,40.0,60.0,44.92003,-93.27488
S10,54,SB,Wed Oct 10 01:15:59 CDT 2018,54.0,40.0,60.0,44.92003,-93.27488
S1148,6,SB,Wed Oct 10 01:15:59 CDT 2018,68.0,0.0,0.0,44.87088,-93.29904
S1148,7,SB,Wed Oct 10 01:15:59 CDT 2018,68.0,0.0,0.0,44.87088,-93.29904


In [25]:
route = route_df.groupby('Station').head(1).index.values


Results = {}


locations = []

for station in route:
    try:
       
        Flow =  float(route_df.loc[station,'Flow'].tail(1)[-1])
        Speed = int(route_df.loc[station,'Speed'].tail(1)[-1])

        Max_Flow = route_df.loc[station,'Flow'].astype(dtype="float")
        Max_Flow = max(Max_Flow.values)
        Max_Speed = route_df.loc[station,'Speed'].astype(dtype="int")
        Max_Speed = max(Max_Speed.values)
        
        lat = float(route_df.loc[station,'Lat'].head(1)[0])
        lng = float(route_df.loc[station,'Lng'].head(1)[0])
        locations.append((lat, lng))
    
        Results[station] = {
            "Current Speed" : Speed,
            "Current Flow" : Flow,
            "Max Speed" : Max_Speed,
            "Max Flow" : Max_Flow,
            "% of Max Speed" : round(Speed / Max_Speed * 100),
            "% of Max Flow" : round(Flow / Max_Flow * 100) ,
            "Lat":lat, 
            "Lng":lng}
        Results = pandas.DataFrame(Results).T
        Results.head()
    except ValueError as v:
        print(f"{station} {v}")


C:\Users\xanen\Anaconda3\lib\site-packages\ipykernel\__main__.py:29: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\xanen\Anaconda3\lib\site-packages\ipykernel\__main__.py:30: RuntimeWarning: invalid value encountered in double_scalars


S10 Length of values does not match length of index
S1148 Length of values does not match length of index
S13 Length of values does not match length of index
S14 Length of values does not match length of index
S15 Length of values does not match length of index
S17 Length of values does not match length of index
S1718 Length of values does not match length of index
S18 Length of values does not match length of index
S19 Length of values does not match length of index
S1943 Length of values does not match length of index
S2 Length of values does not match length of index
S20 Length of values does not match length of index
S21 Length of values does not match length of index
S23 Length of values does not match length of index
S256 Length of values does not match length of index
S3 Length of values does not match length of index
S328 Length of values does not match length of index
S4 Length of values does not match length of index
S43 Length of values does not match length of index
S45 Len

In [24]:
Results

{'44.89021': {'Current Speed': 0,
  'Current Flow': 0.0,
  'Max Speed': 0,
  'Max Flow': 0.0,
  '% of Max Speed': nan,
  '% of Max Flow': nan,
  'Lat': 0.0,
  'Lng': 0.0},
 'S10': {'Current Speed': 40,
  'Current Flow': 60.0,
  'Max Speed': 40,
  'Max Flow': 60.0,
  '% of Max Speed': 100.0,
  '% of Max Flow': 100.0,
  'Lat': 44.92003,
  'Lng': -93.27488},
 'S1148': {'Current Speed': 0,
  'Current Flow': 0.0,
  'Max Speed': 0,
  'Max Flow': 0.0,
  '% of Max Speed': nan,
  '% of Max Flow': nan,
  'Lat': 44.87088,
  'Lng': -93.29904},
 'S13': {'Current Speed': 62,
  'Current Flow': 180.0,
  'Max Speed': 62,
  'Max Flow': 180.0,
  '% of Max Speed': 100.0,
  '% of Max Flow': 100.0,
  'Lat': 44.85991,
  'Lng': -93.29856},
 'S14': {'Current Speed': 0,
  'Current Flow': 0.0,
  'Max Speed': 0,
  'Max Flow': 0.0,
  '% of Max Speed': nan,
  '% of Max Flow': nan,
  'Lat': 44.90227,
  'Lng': -93.27509},
 'S15': {'Current Speed': 0,
  'Current Flow': 0.0,
  'Max Speed': 0,
  'Max Flow': 0.0,
  '% of

In [20]:
folium_map = folium.Map(locations[5],
                        zoom_start=13,
                        tiles="CartoDB positron")
for index, row in Results.iterrows():

    
    folium.CircleMarker(location=(row["Lat"],
                                  row["Lon"]),
                        radius=1,
                        
                        fill=True).add_to(folium_map)

AttributeError: 'dict' object has no attribute 'iterrows'

In [ ]:
Features = []
Last_Sensor = []
for index, row in Results.iterrows():
    Current_Sensor = (row['Lat'], row['Lon'])
    if Last_Sensor == [] :
        Last_Sensor = (row['Lat'], row['Lon'])
    else:
    
        folium.PolyLine([Current_Sensor,Last_Sensor],
         weight=row['Current Occupancy'] * 2.5,
                        popup=f"Speed:{row['Current Speed']} Occupancy: {row['Current Occupancy']}").add_to(folium_map)

        Last_Sensor = (row['Lat'], row['Lon'])





In [ ]:
folium_map


In [ ]:
folium_map.save(f'results/maps/routemap{datetime.datetime.now().strftime("%b%d_%H%M%S")}.html')
print(f'Map saved at results/maps/routemap{datetime.datetime.now().strftime("%b%d_%H%M%S")}.html')